# Import software libraries

In [219]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
- pandas 2.2.1
- sqlite3 3.41.2


In [220]:
if conn:
    conn.close()

# Examine the database

In [221]:
# Connect to SQLite database.
conn = sqlite3.connect('../data/prod_sample.db')


In [222]:
# List all the tables in the database.
query = "SELECT name FROM sqlite_master WHERE type='table'"
pd.read_sql(query,conn)

,name
0,stock_description
1,online_retail_history


# Read data from the `online_retail_history` table

In [223]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = "SELECT * FROM online_retail_history"



# Use the read_sql function in pandas to read a query into a DataFrame.
online_retail_history_df = pd.read_sql(query,conn)


# Preview the first five rows of the data.
online_retail_history_df.head(5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [224]:
# Get the shape of the data.
online_retail_history_df.shape


(15321, 8)

# Read data from the `stock_description` table

In [225]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = "SELECT * FROM stock_description"



# Use the read_sql function in pandas to read a query into a DataFrame.
stock_description_df = pd.read_sql(query,conn)


# Preview the first five rows of the data.
stock_description_df.head(5)


,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [226]:
# Get the shape of the data.
stock_description_df.shape


(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [227]:
stock_description_df.head()

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [228]:
online_retail_history_df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [229]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.
query = """SELECT o.*,s.Description FROM
           online_retail_history AS o LEFT JOIN stock_description AS s
           ON o.StockCode = s.StockCode"""





# Use the read_sql function in pandas to read a query into a DataFrame.
aggregated_df = pd.read_sql(query,conn)


# Preview the first five rows of the data.
aggregated_df.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


In [230]:
# Get the shape of the data.
aggregated_df.shape


(17032, 9)

# Identify and fix corrupt or unusable data

In [231]:
aggregated_df.Country.value_counts()

Country
United Kingdom    17032
Name: count, dtype: int64

In [232]:
aggregated_df.describe()

,Quantity,Price,TotalAmount
count,17032.000000,17019.000000,17019.000000
mean,15.649072,5.170809,44.754389
std,75.335342,5.339212,137.353757
min,1.000000,0.400000,0.550000
25%,2.000000,1.650000,9.900000
50%,5.000000,2.950000,16.500000
75%,12.000000,4.960000,35.400000
max,4300.000000,32.040000,4921.500000


In [233]:
# Check the value counts of the "Description" field.
aggregated_df.Description.value_counts()


Description
CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
?                                     1711
REGENCY CAKESTAND 3 TIER              1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: count, dtype: int64

There are not any anomalies for now except "?" value at description column. We should further analyze it.

In [234]:
aggregated_df[aggregated_df['Description'] == '?']

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
36,536477,22423,16,2010-12-01 12:27:00,10.95,u1621,United Kingdom,175.20,?
40,536502,22423,2,2010-12-01 12:36:00,12.75,u16552,United Kingdom,25.50,?
45,536525,22423,2,2010-12-01 12:54:00,12.75,u14078,United Kingdom,25.50,?
48,536528,22423,1,2010-12-01 13:17:00,12.75,u15525,United Kingdom,12.75,?
50,536530,22423,1,2010-12-01 13:21:00,12.75,u17905,United Kingdom,12.75,?
...,...,...,...,...,...,...,...,...,...
16993,581449,22423,1,2011-12-08 17:37:00,12.75,u12748,United Kingdom,12.75,?
16995,581449,22423,1,2011-12-08 17:37:00,12.75,u12748,United Kingdom,12.75,?
17004,581472,22423,2,2011-12-08 19:55:00,12.75,u15796,United Kingdom,25.50,?
17016,581495,22423,10,2011-12-09 10:20:00,12.75,u14051,United Kingdom,127.50,?


In [235]:
aggregated_df[aggregated_df['Description'] == '?']["StockCode"].unique()

array(['22423'], dtype=object)

In [236]:
aggregated_df[aggregated_df["StockCode"] == '22423']["Description"].unique()

array(['?', 'REGENCY CAKESTAND 3 TIER'], dtype=object)

If there is a one-to-one match between stock codes and descriptions we can impute "regency cakstand 3 tier" for "?".First we make imputing to check it.

In [237]:
aggregated_df_test = aggregated_df.copy()
aggregated_df_test.loc[aggregated_df_test['Description'] == '?', 'Description'] = "REGENCY CAKESTAND 3 TIER"

In [238]:
aggregated_df_test["match"] = aggregated_df_test["StockCode"] + " - " + aggregated_df_test["Description"]
match = aggregated_df_test["match"]
print(match.nunique() == aggregated_df_test.StockCode.nunique() == aggregated_df_test.Description.nunique(),"-",match.nunique())

True - 10


Done. There is a one to one match. So we can make imputation.

In [239]:
aggregated_df_test.drop("match",axis=1,inplace=True)
aggregated_df = aggregated_df_test
del aggregated_df_test

# Identify and remove duplicates

In [240]:
print("#rows before: ", aggregated_df.shape[0])
print("dropped ",aggregated_df.duplicated().sum()," rows")
aggregated_df.drop(aggregated_df[aggregated_df.duplicated()].index,axis=0,inplace=True)
print("#rows now: ", aggregated_df.shape[0])

#rows before:  17032
dropped  1826  rows
#rows now:  15206


# Correct date formats

In [241]:
# Get the data types for every column in the DataFrame.

aggregated_df.dtypes

Invoice         object
StockCode       object
Quantity         int64
InvoiceDate     object
Price          float64
CustomerID      object
Country         object
TotalAmount    float64
Description     object
dtype: object

In [242]:
import datetime as dt

In [243]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.
aggregated_df['InvoiceDate'] = pd.to_datetime(aggregated_df['InvoiceDate']).dt.date



In [244]:
aggregated_df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Examine the table before finishing

In [245]:
# Preview the first five rows of the data.
aggregated_df.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Load the dataset into a pickle file

In [246]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
aggregated_df.to_pickle('../data/online_history_cleaned.pickle')


In [247]:
# Close any connections to the database.
conn.close()
